## Datensätze zu einem verbinden

In [31]:
import pandas as pd

# Schritt A: World Bank Population Daten einlesen
pop_raw = pd.read_csv("worldbank_population_raw.csv")

# Wir behalten nur die relevanten Spalten
pop_clean = pop_raw[["countryiso3code", "date", "value"]].copy()

# Spalten umbenennen
pop_clean.rename(columns={
    "countryiso3code": "iso3",
    "date": "Year",
    "value": "Population"
}, inplace=True)

# Datentypen anpassen (Year als Integer)
pop_clean["Year"] = pop_clean["Year"].astype(int)

# Nur Jahre ab 1960 verwenden (weil die World Bank erst ab 1960 Daten liefert)
pop_clean = pop_clean[pop_clean["Year"] >= 1960].reset_index(drop=True)

pop_clean


,iso3,Year,Population
0,ARG,2014,43024071
1,ARG,2013,42582455
2,ARG,2012,42161721
3,ARG,2011,41730660
4,ARG,2010,41288694
...,...,...,...
930,USA,1964,191889000
931,USA,1963,189242000
932,USA,1962,186538000
933,USA,1961,183691000


In [32]:
# Schritt B: WorldCups Datensatz einlesen
cups = pd.read_csv("WorldCups.csv")

cups.head()


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607


In [33]:
country_to_iso3 = {
    "Argentina": "ARG",
    "USA": "USA",
    "Uruguay": "URY",
    "Germany": "DEU",
    "Italy": "ITA",
    "Brazil": "BRA",
    "Hungary": "HUN",
    "Sweden": "SWE",
    "Austria": "AUT",
    "France": "FRA",
    "Chile": "CHL",
    "England": "GBR",   # Achtung: England → GBR (UK)
    "Portugal": "PRT",
    "Netherlands": "NLD",
    "Poland": "POL",
    "Croatia": "HRV",
    "Turkey": "TUR",
    "Spain": "ESP" 
}


In [34]:
cups["Winner_iso3"] = cups["Winner"].map(country_to_iso3)
cups["RunnersUp_iso3"] = cups["Runners-Up"].map(country_to_iso3)
cups["Third_iso3"] = cups["Third"].map(country_to_iso3)

cups.head()


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,Winner_iso3,RunnersUp_iso3,Third_iso3
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549,URY,ARG,USA
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000,ITA,NaN,DEU
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700,ITA,HUN,BRA
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246,URY,BRA,SWE
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607,NaN,HUN,AUT


In [35]:
cups[cups[["Winner_iso3", "RunnersUp_iso3", "Third_iso3"]].isna().any(axis=1)]


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,Winner_iso3,RunnersUp_iso3,Third_iso3
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000,ITA,NaN,DEU
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607,NaN,HUN,AUT
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893.172,BRA,NaN,CHL
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1.563.135,GBR,NaN,PRT
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1.603.975,BRA,ITA,NaN
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1.865.753,NaN,NLD,POL
11,1982,Spain,Italy,Germany FR,Poland,France,146,24,52,2.109.723,ITA,NaN,POL
12,1986,Mexico,Argentina,Germany FR,France,Belgium,132,24,52,2.394.031,ARG,NaN,FRA
13,1990,Italy,Germany FR,Argentina,Italy,England,115,24,52,2.516.215,NaN,ARG,ITA


In [36]:
# Sonderfälle in den Country-Namen bereinigen

# 1. Deutschland vereinheitlichen: immer "Germany"
normalize_names = {
    "Germany FR": "Germany",
    "Germany DR": "Germany"
}

for col in ["Winner", "Runners-Up", "Third"]:
    cups[col] = cups[col].replace(normalize_names)

# 2. Länder, die nicht mehr existieren, aus dem Datensatz entfernen
obsolete_countries = ["Czechoslovakia", "Soviet Union", "Yugoslavia"]

cups = cups[
    ~cups["Winner"].isin(obsolete_countries)
    & ~cups["Runners-Up"].isin(obsolete_countries)
    & ~cups["Third"].isin(obsolete_countries)
].copy()

cups



,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,Winner_iso3,RunnersUp_iso3,Third_iso3
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549,URY,ARG,USA
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700,ITA,HUN,BRA
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246,URY,BRA,SWE
4,1954,Switzerland,Germany,Hungary,Austria,Uruguay,140,16,26,768.607,NaN,HUN,AUT
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819.810,BRA,SWE,FRA
7,1966,England,England,Germany,Portugal,Soviet Union,89,16,32,1.563.135,GBR,NaN,PRT
8,1970,Mexico,Brazil,Italy,Germany,Uruguay,95,16,32,1.603.975,BRA,ITA,NaN
9,1974,Germany,Germany,Netherlands,Poland,Brazil,97,16,38,1.865.753,NaN,NLD,POL
10,1978,Argentina,Argentina,Netherlands,Brazil,Italy,102,16,38,1.545.791,ARG,NLD,BRA
11,1982,Spain,Italy,Germany,Poland,France,146,24,52,2.109.723,ITA,NaN,POL


In [37]:
cups["Winner_iso3"]      = cups["Winner"].map(country_to_iso3)
cups["RunnersUp_iso3"]   = cups["Runners-Up"].map(country_to_iso3)
cups["Third_iso3"]       = cups["Third"].map(country_to_iso3)

# Kontrolle: gibt es noch NaN-ISO-Codes?
cups[cups[["Winner_iso3", "RunnersUp_iso3", "Third_iso3"]].isna().any(axis=1)]


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,Winner_iso3,RunnersUp_iso3,Third_iso3


In [38]:
# Jahr-Spalte im WorldCups-Datensatz als Integer
cups["Year"] = cups["Year"].astype(int)

# Nur World Cups ab 1960 berücksichtigen (passt zur World-Bank-Population)
cups = cups[cups["Year"] >= 1960].reset_index(drop=True)

cups[["Year", "Winner", "Winner_iso3", "Runners-Up", "RunnersUp_iso3", "Third", "Third_iso3"]].head()


,Year,Winner,Winner_iso3,Runners-Up,RunnersUp_iso3,Third,Third_iso3
0,1966,England,GBR,Germany,DEU,Portugal,PRT
1,1970,Brazil,BRA,Italy,ITA,Germany,DEU
2,1974,Germany,DEU,Netherlands,NLD,Poland,POL
3,1978,Argentina,ARG,Netherlands,NLD,Brazil,BRA
4,1982,Italy,ITA,Germany,DEU,Poland,POL


## die datensätze verbinden

In [39]:
# Schritt 1: Population der Gewinner an den WorldCups-Datensatz mergen

# 1) Kopie von pop_clean vorbereiten, die zu den Winner-Spalten passt
winner_pop = pop_clean.rename(columns={
    "iso3": "Winner_iso3",
    "Population": "Winner_population"
})

# 2) Merge: cups (links) mit winner_pop (rechts) über Year + Winner_iso3
cups = cups.merge(
    winner_pop[["Winner_iso3", "Year", "Winner_population"]],
    on=["Winner_iso3", "Year"],
    how="left"
)

# 3) Ergebnis kurz inspizieren
cups[["Year", "Winner", "Winner_iso3", "Winner_population"]].head()


,Year,Winner,Winner_iso3,Winner_population
0,1966,England,GBR,54648500.0
1,1970,Brazil,BRA,95375651.0
2,1974,Germany,DEU,78967433.0
3,1978,Argentina,ARG,27132158.0
4,1982,Italy,ITA,56543548.0


In [40]:
# Schritt 2: Population der Runners-Up zum Datensatz hinzufügen

# 1) Kopie von pop_clean für die Runners-Up vorbereiten
runners_pop = pop_clean.rename(columns={
    "iso3": "RunnersUp_iso3",
    "Population": "RunnersUp_population"
})

# 2) Merge: cups (links) mit runners_pop (rechts) über Year + RunnersUp_iso3
cups = cups.merge(
    runners_pop[["RunnersUp_iso3", "Year", "RunnersUp_population"]],
    on=["RunnersUp_iso3", "Year"],
    how="left"
)

# 3) Ergebnis ansehen
cups[["Year", "Runners-Up", "RunnersUp_iso3", "RunnersUp_population"]].head()


,Year,Runners-Up,RunnersUp_iso3,RunnersUp_population
0,1966,Germany,DEU,76600311
1,1970,Italy,ITA,53821850
2,1974,Netherlands,NLD,13545056
3,1978,Netherlands,NLD,13941700
4,1982,Germany,DEU,78333366


In [41]:
# Schritt 3: Population der Drittplatzierten zum Datensatz hinzufügen

# 1) Kopie von pop_clean für Third vorbereiten
third_pop = pop_clean.rename(columns={
    "iso3": "Third_iso3",
    "Population": "Third_population"
})

# 2) Merge: cups (links) mit third_pop (rechts) über Year + Third_iso3
cups = cups.merge(
    third_pop[["Third_iso3", "Year", "Third_population"]],
    on=["Third_iso3", "Year"],
    how="left"
)

# 3) Ergebnis ansehen (kleine Vorschau)
cups[[
    "Year",
    "Third",
    "Third_iso3",
    "Third_population"
]].head()


,Year,Third,Third_iso3,Third_population
0,1966,Portugal,PRT,8930990
1,1970,Germany,DEU,78169289
2,1974,Poland,POL,33678899
3,1978,Brazil,BRA,115600942
4,1982,Poland,POL,36230481


In [42]:
# Schritt 4: Gesamt-Vorschau des angereicherten WorldCups-Datensatzes

cols_to_show = [
    "Year",
    "Winner", "Winner_iso3", "Winner_population",
    "Runners-Up", "RunnersUp_iso3", "RunnersUp_population",
    "Third", "Third_iso3", "Third_population"
]

cups[cols_to_show].head(10)


,Year,Winner,Winner_iso3,Winner_population,Runners-Up,RunnersUp_iso3,RunnersUp_population,Third,Third_iso3,Third_population
0,1966,England,GBR,54648500.0,Germany,DEU,76600311,Portugal,PRT,8930990
1,1970,Brazil,BRA,95375651.0,Italy,ITA,53821850,Germany,DEU,78169289
2,1974,Germany,DEU,78967433.0,Netherlands,NLD,13545056,Poland,POL,33678899
3,1978,Argentina,ARG,27132158.0,Netherlands,NLD,13941700,Brazil,BRA,115600942
4,1982,Italy,ITA,56543548.0,Germany,DEU,78333366,Poland,POL,36230481
5,1986,Argentina,ARG,30811980.0,Germany,DEU,77720436,France,FRA,56956002
6,1990,Germany,DEU,79433029.0,Argentina,ARG,32755901,Italy,ITA,56719240
7,1994,Brazil,BRA,159265006.0,Italy,ITA,56843400,Sweden,SWE,8780745
8,1998,France,FRA,60190684.0,Brazil,BRA,169159655,Croatia,HRV,4532135
9,2002,Brazil,BRA,178503484.0,Germany,DEU,82488495,Turkey,TUR,67048795


In [43]:
cups[cups[["Winner_population", "RunnersUp_population", "Third_population"]].isna().any(axis=1)][cols_to_show]


,Year,Winner,Winner_iso3,Winner_population,Runners-Up,RunnersUp_iso3,RunnersUp_population,Third,Third_iso3,Third_population
11,2010,Spain,ESP,NaN,Netherlands,NLD,16615394,Germany,DEU,81776930
